In [1]:
import math
import datetime
import os, sys
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from IPython.display import Audio
import csv

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torchsummary import summary

import cv2




# local files
sys.path.insert(0, '../')
import pyClient
import utils
import model
from model import Transition

In [2]:
# Training parameters
BATCH_SIZE = 128 #original 128
GAMMA = 0.999
EPS_START = 0.95
EPS_END = 0.05
EPS_DECAY_steps = 4000 
EPS_DECAY = (EPS_START - EPS_END)/EPS_DECAY_steps
REPLAY_START_SIZE =  1500
TARGET_UPDATE = 10 #episodes
DEVICE = 'cuda:0'
LR_DQN = 0.01



# Environment parameters
IMSIZE = 128
STACK_SIZE = 4
N_ACTIONS = 3
IP  = "127.0.0.1" # Ip address that the TCP/IP interface listens to
PORT = 13000       # Port number that the TCP/IP interface listens to


environment = pyClient.Environment(ip = IP, port = PORT, size = IMSIZE) 
agent = model.DoubleDQNAgent(imsize=IMSIZE,
                 in_channels=STACK_SIZE,
                 n_actions=N_ACTIONS,
                 memory_capacity=12000,
                 eps_start=EPS_START,
                 eps_end=EPS_END,
                 eps_delta=EPS_DECAY,
                 gamma_discount = GAMMA,
                 batch_size = BATCH_SIZE,
                 device=DEVICE)


optimizer = optim.Adam(agent.policy_net.parameters(), lr = LR_DQN)

In [3]:
# Utilities
def process_frame(state_raw):
    """ @TODO 
    - Image processing
    - Phosphene simulation
    - Frame stacking
    """
    frame = environment.state2usableArray(state_raw)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = frame.astype('float32')
    return torch.Tensor(frame / 255.).view(1,1,environment.size, environment.size)

def validation_loop(agent,environment,img_processing, val_seeds=[251,252,253,254,255]):
    # How to handle the different end signals
    RESET_UPON_END_SIGNAL = {0:False,  # Nothing happened
                             1:False,   # Box collision
                             2:False,   # Wall collision
                             3:True}    # Reached step target
    RESET_AFTER_NR_SIDESTEPS = 5
    
    # Set nn.module to evaluation mode
    agent.policy_net.eval()
    
    # Reset counters 
    wall_collisions = 0
    box_collisions = 0
    total_reward = 0
    endless_loops = 0
    step_count = 0
    

    for seed in val_seeds:

        # Reset environment at start of episode
        _, _, _ = environment.setRandomSeed(seed)
        _, _, _ = environment.reset(environment.training_condition)

        
        # Create an empty frame stack and fill it with frames
        frame_stack = utils.FrameStack(stack_size=STACK_SIZE)
        for _ in range(STACK_SIZE):
            _, _, frame_raw = environment.step(0)
            frame = img_processing(frame_raw).to(agent.device) 
            state = frame_stack.update_with(frame)
        
        side_steps = 0

        # Episode starts here:
        for t in count(): 

            # 1. Agent performs a step (based on the current state) and obtains next state
            action = agent.select_action(state,validation=True)
            end, reward, next_state_raw = environment.step(action.item())
            frame = img_processing(next_state_raw).to(agent.device) 
            next_state = frame_stack.update_with(frame)
            side_steps = side_steps + 1  if action != 0 else 0
            
            # 2. interpret reward
            if reward > 100:
                reward = -(reward -100)

            # 3. Store performance and training measures
            total_reward += reward;
            if end == 1:
                box_collisions += 1
            if end == 2:
                wall_collisions +=1

            # 4. the episode ends here after reaching step target or too many side steps
            if side_steps>RESET_AFTER_NR_SIDESTEPS:
                endless_loops += 1
                step_count += t
                break
            elif RESET_UPON_END_SIGNAL[end]:
                step_count += t
                break
            else:
                state = next_state
                
    agent.policy_net.train()
    print('step count {} wall_collisions: {}, box_collisions: {}, endless_loops: {}, total_reward: {}'.format(step_count, wall_collisions, box_collisions, endless_loops, total_reward))
    return step_count, wall_collisions, box_collisions, endless_loops, total_reward

In [ ]:
## Some more training parameters 
MAX_EPISODES = 500 # number of episodes (an episode ends after agent hits a box)
MAX_STEPS  = 5e4  # number of optimization steps (each time step the model parameters are updated)
TRAINING_CONDITION = 0 # 0: plain training, 1: complex training, 2: plain testing 3: complex testing
LOGFILE = 'Out/test-09.csv'
MODEL_PATH = 'Out/test-09.pth'
SEED = 0

# How to handle the different end signals
RESET_UPON_END_SIGNAL = {0:False,  # Nothing happened
                         1:True,   # Box collision
                         2:False,   # Wall collision
                         3:True}  # Reached step target
RESET_AFTER_NR_SIDESTEPS = 5


torch.manual_seed(SEED)
environment.training_condition = TRAINING_CONDITION

# Write header to logfile 
with open(LOGFILE, 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['episode','step_count', 
                     'wall_collisions', 'box_collisions',
                     'endless_loops','reward', 'train_loss', 'validation'])


# Counters 
wall_collisions = 0
box_collisions = 0
episode_reward = 0
endless_loops = 0
total_loss = 0
step_count = 0
best_reward = 0



for episode in range(int(MAX_EPISODES)):
    
    # @TODO: SAVE AFTER VALIDATION
    if episode % 50 == 0:
        val_performance = validation_loop(agent,environment,process_frame)
        val_reward = val_performance[-1]
        if val_reward > best_reward:
            print("new best model")
            best_reward = val_reward
            torch.save(agent.policy_net.state_dict(), MODEL_PATH)
            
        # Write validation performance to log file
        with open(LOGFILE, 'a') as csvfile:
            writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            writer.writerow([episode, *val_performance, 0, 1])        

    # Write training performance to log file
    with open(LOGFILE, 'a') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([episode,step_count,
                         wall_collisions, box_collisions, 
                         endless_loops, episode_reward,total_loss, 0]) 
    
    # Reset counters 
    wall_collisions = 0
    box_collisions = 0
    episode_reward = 0
    endless_loops = 0
    step_count = 0

    # Stop training after 
    if agent.step_count > MAX_STEPS:
        break
    
    # Side-step counter (to prevent endless loops)
    side_steps = 0
    
    
    # Target net is updated once in a few episodes (double Q-learning)
    if episode % TARGET_UPDATE == 0:  #episodes
        print('episode {}, target net updated'.format(episode))
        agent.update_target_net
    
    
    # Reset environment at start of episode
    seed = torch.randint(250,(1,)).item()
    _, _, _ = environment.setRandomSeed(seed)
    _, _, _ = environment.reset(environment.training_condition)
    
    # Create an empty frame stack and fill it with frames
    frame_stack = utils.FrameStack(stack_size=STACK_SIZE)
    for _ in range(STACK_SIZE):
        _, _, frame_raw = environment.step(0)
        frame = process_frame(frame_raw).to(DEVICE) 
        state = frame_stack.update_with(frame)
    
    # Episode starts here:
    for t in count(): 
        
        # 1. Agent performs a step (based on the current state) and obtains next state
        action = agent.select_action(state)
        side_steps = side_steps + 1  if action != 0 else 0
        end, reward, frame_raw = environment.step(action.item())
        agent_died = RESET_UPON_END_SIGNAL[end] or side_steps > RESET_AFTER_NR_SIDESTEPS
        frame = process_frame(frame_raw).to(DEVICE)
        next_state = frame_stack.update_with(frame) if not agent_died else None
        
        # 2. Interpret reward signal
        if reward > 100:
            reward = -(reward -100)
        
        # 3. Push the transition to replay memory (in the right format & shape)
        reward = torch.tensor([reward], device=DEVICE,dtype=torch.float)
        action = action.unsqueeze(0)
        agent.memory.push(state, action, next_state, reward)
        
    
        # 4. optimize model
        if len(agent.memory) > REPLAY_START_SIZE:
            
            state_action_values, expected_state_action_values = agent.forward()
            
            # Compute Huber loss
            loss = F.smooth_l1_loss(state_action_values, expected_state_action_values)
            total_loss += loss.item()

            # Optimize the model
            optimizer.zero_grad()
            loss.backward()

            # Gradient clipping
            nn.utils.clip_grad_norm_(agent.policy_net.parameters(), 1)

            # Update the model parameters
            optimizer.step()
            
            """ @TODO: validation loop"""
        else:
            # Do not count as optimization loop
            agent.step_count = 0
            
        # 5. Store performance and training measures
        step_count += 1
        episode_reward += reward.item();
        if end == 1:
            box_collisions += 1
        if end == 2:
            wall_collisions +=1
        if side_steps > RESET_AFTER_NR_SIDESTEPS:    
            endless_loops +=1

        
        
        # 6. the episode ends here if agent performed any 'lethal' action (specified in RESET_UPON_END_SIGNAL)
        if agent_died:
            break
        else:
            state = next_state

step count 104 wall_collisions: 38, box_collisions: 3, endless_loops: 5, total_reward: 31
new best model
episode 0, target net updated
episode 10, target net updated
episode 20, target net updated
episode 30, target net updated
episode 40, target net updated
step count 25 wall_collisions: 8, box_collisions: 0, endless_loops: 5, total_reward: -102
episode 50, target net updated


In [ ]:
EXAMPLES = 10

i = 0

# for state, action, next_state, reward in agent.memory.memory[:EXAMPLES]:
for state, action, next_state, reward in bad_choices[:EXAMPLES]:
    i+=1
    
    plt.figure(figsize = (10,10), dpi=200)
    img = torch.cat([state[0,t,...] for t in range(STACK_SIZE)],dim=1)
    if next_state is not None:
        img = torch.cat([img, next_state[0,-1,...]],dim=1)
        plt.axvline(x=STACK_SIZE*IMSIZE,color='r')
    plt.imshow(img.detach().cpu().numpy())
    plt.title('Action: {}, Reward {}'.format(action.item(),reward.item()))
    plt.axis('off')
    plt.ylabel('frames >')
    plt.xlabel('state | next state')
    plt.show()
    
# plt.tight_layout()


In [ ]:
bad_choices = [(state, action, next_state,reward) for (state, action, next_state,reward) in agent.memory.memory if reward<0 and action ==0]

In [ ]:
len(bad_choices)